In [1]:
from proxy_chrome_browser import get_chromedriver, build_proxy_extension
import moviepy.editor as mpy
import glob
import os
from time import sleep
from tqdm import tqdm

In [2]:
br = get_chromedriver(headless=True,path='/home/miko/Downloads/chromedriver-v-106-0-5249-61')

In [3]:
def find_elm(tag,driver=br,_all=False,_by="css"):
    from selenium.webdriver.common.by import By
    if _by=="css":
        _by=By.CSS_SELECTOR
    elif _by=="link_text":
        _by=By.PARTIAL_LINK_TEXT
    elif _by=="link_text_exact":
        _by=By.LINK_TEXT
        
    try:
        if _all:
            out = driver.find_elements(by=_by,value=tag)
        else:
            out = driver.find_element(by=_by,value=tag)
    except:
        #print('None')
        out = None
    return out

In [4]:
def save_screenshot(elm,filename):
    assert filename[-3:] == "png", "filename should end with png"
    with open(filename,"wb") as f:
        f.write(elm.screenshot_as_png)

In [5]:
def scroll_into_view(elm):
     #source: -https://stackoverflow.com/questions/25733877/selenium-how-do-you-check-scroll-position
     #loc = elm.location
     #current_view_script = """ var doc = document.documentElement;
     #var left = (window.pageXOffset || doc.scrollLeft) - (doc.clientLeft || 0);
     #var top = (window.pageYOffset || doc.scrollTop)  - (doc.clientTop || 0);
     #taking screenshot will scroll into view. IF this stops working can use javascript above
     elm.screenshot_as_png
     sleep(.5)


In [6]:
#set to dark theme
br.get("https://www.reddit.com")

In [7]:

icon = find_elm("button[id='USER_DROPDOWN_ID']")
icon.click()
sleep(.5)
dark_btn = find_elm("button[data-redditstyle='true']")
dark_btn.click()
sleep(.5)
# if for some reason click isnt working. set it manually
#input("Confirm darkmode has been set (press enter)")

In [8]:
post_url = "https://www.reddit.com/r/AskReddit/comments/ym1lxv/reddit_whats_your_most_im_with_the_boomers_on/"

In [9]:
br.get(post_url)

In [10]:
#br.switch_to.active_element.text

In [12]:
# expand replies
num_replies_threshold = 90
get_replies = True
num_total_loops = 5
current_loop_num = 0
clicked_links = []
while get_replies:
    more_replies_divs = find_elm("div[id^='moreComments']",_all=True)
    expandable_divs = []
    for div in more_replies_divs:
        more_replies_link = find_elm("p",driver=div)
        #p = br.execute_script("return arguments[0].parentNode;",more_replies_link)
        if not more_replies_link:
            continue
        if more_replies_link.text == '' or 'loading..' in more_replies_link.text:
            continue
        num_replies = int(more_replies_link.text.split()[0].replace(",",""))
        if num_replies >= num_replies_threshold:
            if more_replies_link not in clicked_links:
                expandable_divs.append(more_replies_link)
            else:
                continue
    for div in expandable_divs:
        #link_elm = find_elm("div[data-testid^='search-scope-pill-text']")
        try:
            scroll_into_view(more_replies_link)
            div.click()
            sleep(1)
        except:
            continue  
    if len(expandable_divs) == 0:
        get_replies = False
    if current_loop_num >= num_total_loops:
        get_replies = False
    else:
        current_loop_num += 1
    print("getting more replies")
    #todo: find a better stopping condition

getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


In [13]:
# get comments
comments = find_elm("div[class*='Comment']",_all=True)

In [14]:
len(comments)

590

In [15]:
submission_title = br.current_url.split("/")[-2]
#asset_dir="assets/demo_png_2_vid"
asset_dir="/mnt/n/data/reddit-yt-tts/queued"
completed_dir="/mnt/n/data/reddit-yt-tts/completed"
#thread_ids = []

In [16]:
def save_comment_text(filename,comment):
    comment_text = " ".join([x.text for x in find_elm("p",driver=comment,_all=True)])
    
    with open(filename,"w") as f:
        f.write(comment_text)

In [17]:
#save threaded comment screenshot as png to directory
vote_count_threshold = 100
thread_count = 0
for comment in tqdm(comments):
    # filter deleted comments
    if "Comment deleted by user" in comment.text:
        continue
    
    # some comments arnt expanded, skipping
    if len(comment.text.split("\n")) <= 4:
           continue
           
    #filter highly voted comments
    vote_div = find_elm("div[id^='vote-arrows']",driver=comment)
    multiply_1k = 'k' in vote_div.text
    if multiply_1k:
        vote_count = float(vote_div.text.replace("k","")) * 1000
    else:
        vote_count = int(vote_div.text)
    
    comment_lvl = find_elm("span",comment).text
    
    if "1" in comment_lvl:
        thread_count += 1
        user = comment.text.split("\n")[1]
        vote_count_txt = vote_div.text.replace(".","_")
        thread_id = f"{thread_count:04}-{user}-{vote_count_txt}"
        
        comment_id=0
    else:
        comment_id+=1
    
    save_comment = False
    if vote_count >= vote_count_threshold:
        save_comment = True
    
    if save_comment:
        #thread_ids.append(thread_id)
        folder = f"{asset_dir}/{submission_title}/{thread_id}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        save_screenshot(comment,f"{folder}/{comment_id:04}.png")
        save_comment_text(filename=f"{folder}/{comment_id:04}.txt",comment=comment)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 590/590 [03:48<00:00,  2.59it/s]


In [18]:
thread_ids = [x.split("/")[-1] for x in glob.glob(f"{asset_dir}/{submission_title}/*")]

In [19]:
def build_thread_vid(thread_id):
    images = glob.glob(f"{asset_dir}/{submission_title}/{thread_id}/*.png")
    audio_file_dir = f"{completed_dir}/{submission_title}/{thread_id}"
    # https://zulko.github.io/moviepy/examples/examples.html
    clips = []
    audio_clips = []
    total_duration = 0
    transition_duration = 0
    for i,image in enumerate(images):
        image_file = image.split("/")[-1]
        audio_file = image_file.replace(".png",".wav")
        try:
            audio_clip = mpy.AudioFileClip(f"{audio_file_dir}/{audio_file}").set_start(total_duration)
        except OSError as e:
            print(f"audio file {audio_file_dir}/{audio_file} not found")
            continue
        audio_clips.append(audio_clip)
        audio_duration = audio_clip.duration
        
        clip = mpy.ImageClip(image).set_duration(audio_duration).set_start(total_duration)
    
        
        #clip = clip.set_start(5) # TODO: change duration based on audio
        #clip = clip.resize((1080,1920)) #TODO add tts audio
        clips.append(clip)
        
        total_duration += audio_duration + transition_duration
  
    if len(clips) > 0 :
        final_audio_clip = mpy.CompositeAudioClip(audio_clips)
        final_clip = mpy.CompositeVideoClip(clips)
        final_clip.audio = final_audio_clip
        final_clip.write_videofile(f"{asset_dir}/{submission_title}/{thread_id}/thread_clip.mp4",fps=30,logger=None)
    

In [20]:
f"{asset_dir}/{submission_title}/{thread_ids[0]}"

'/mnt/n/data/reddit-yt-tts/queued/reddit_whats_your_most_im_with_the_boomers_on/0005-EternityLeave-24_1k'

In [21]:
comments[0].text

'level 1\nUnique_Football_8839\n·\n6 days ago\nTouchscreens in cars are a fundamentally bad idea & physical buttons (or similar controls) need to be brought back.\nDistracted driving is at an all-time high. Touchscreen controls literally require you to take your eyes off the road and can create a longer time of inattention due to their complexity.\nI have an old-ass car with physical controls. I can do dang near anything (radio, climate control, etc.) without taking my eyes off the road and in much less time.\n1.1k\nReply\nShare\nReport\nSave\nFollow'

In [25]:
# Execute TTS at this point (on windows GPU) then run below
input("Execute TTS at this point (on windows GPU) then run below...(press enter)")

Execute TTS at this point (on windows GPU) then run below... 


''

In [26]:
# save threads to clips
# vote_div.text
thread_ids = list(set(thread_ids))
for thread in tqdm(thread_ids):
    print(thread)
    build_thread_vid(thread)

  0%|                                                                                                                                                          | 0/195 [00:00<?, ?it/s]

0101-Ancient_Wisdom_Yall-53_8k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0101-Ancient_Wisdom_Yall-53_8k/0006.wav not found
0066-Dragoonscaper-7_7k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0066-Dragoonscaper-7_7k/0001.wav not found
0121-ujythrsgfdd-20_6k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0121-ujythrsgfdd-20_6k/0003.wav not found
0025-Pollowollo-37_3k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0025-Pollowollo-37_3k/0002.wav not found
0012-EternityLeave-24_1k


  7%|█████████▋                                                                                                                                       | 13/195 [00:01<00:16, 10.95it/s]

0085-buckyhermit-4_1k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0085-buckyhermit-4_1k/0001.wav not found
0064-Ghostleeee-55_4k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0064-Ghostleeee-55_4k/0004.wav not found
0102-SoulExecution-1_5k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0102-SoulExecution-1_5k/0000.wav not found
0074-iamapizza-15_2k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0074-iamapizza-15_2k/0001.wav not found
0161-nithrilh-144
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0161-nithrilh-144/0000.wav not found
0035-claytondb-27_9k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0035-claytondb-27_9k/0004.wav not found
0103-hokumpocus-2_7k
audio file /mnt/n/data/reddit-yt-tts/completed/redd

 11%|███████████████▌                                                                                                                                 | 21/195 [00:01<00:09, 19.20it/s]

...

 22%|███████████████████████████████▉                                                                                                                 | 43/195 [00:04<00:13, 11.01it/s]

audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0044-Kanedi4s-15_4k/0000.wav not found
0077-Fit-Rest-973-1_3k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0077-Fit-Rest-973-1_3k/0000.wav not found
0119-ujythrsgfdd-20_6k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0119-ujythrsgfdd-20_6k/0001.wav not found
0091-RagnarStonefist-2_2k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0091-RagnarStonefist-2_2k/0000.wav not found
0060-Ghostleeee-55_4k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0060-Ghostleeee-55_4k/0000.wav not found
0184-realmartwo-333
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0184-realmartwo-333/0001.wav not found
0162-Unsimulated-458
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats

 25%|████████████████████████████████████▍                                                                                                            | 49/195 [00:08<00:26,  5.55it/s]

0173-ThePhoenixBird2022-315
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0173-ThePhoenixBird2022-315/0000.wav not found
0022-SpartanR259-3_4k
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0022-SpartanR259-3_4k/0001.wav not found
0151-Ghost_Star326-100
audio file /mnt/n/data/reddit-yt-tts/completed/reddit_whats_your_most_im_with_the_boomers_on/0151-Ghost_Star326-100/0000.wav not found
0006-anonymouslyinvisible-34_1k


KeyboardInterrupt: 

In [ ]:
# save c to 

In [134]:
videos = glob.glob(f"{asset_dir}/{submission_title}/*/*.mp4")

In [135]:
all_video_clips = [mpy.VideoFileClip(video) for video in videos]

In [136]:
videos_w_sound_corrected = []
for clip in all_video_clips:
    clip.audio = clip.audio.fx(mpy.afx.audio_normalize)
    videos_w_sound_corrected.append(clip)

In [137]:
final_video = mpy.concatenate_videoclips(videos_w_sound_corrected,method="compose")

In [138]:
#import moviepy.video.fx as vfx
#import moviepy.audio.fx as afx
bg_music = mpy.AudioFileClip("assets/bg-test.mp3")

bg_music = bg_music.fx( mpy.afx.audio_normalize).fx( mpy.afx.volumex, 0.3).fx( mpy.afx.audio_fadein, 1.0)
tts_audio = final_video.audio

In [139]:
bgm_w_tts_audio = mpy.CompositeAudioClip([bg_music,tts_audio])

In [140]:
final_video.audio = bgm_w_tts_audio

In [141]:
final_video.write_videofile(f"assets/test_final_vid.mp4",fps=30,logger=None)

In [142]:
1+1

2

In [87]:
mpy.concatenate_videoclips?

Signature:
mpy.concatenate_videoclips(
    clips,
    method='chain',
    transition=None,
    bg_color=None,
    ismask=False,
    padding=0,
)
Docstring:
Concatenates several video clips

Returns a video clip made by clip by concatenating several video clips.
(Concatenated means that they will be played one after another).

There are two methods:

- method="chain": will produce a clip that simply outputs
  the frames of the succesive clips, without any correction if they are
  not of the same size of anything. If none of the clips have masks the
  resulting clip has no mask, else the mask is a concatenation of masks
  (using completely opaque for clips that don't have masks, obviously).
  If you have clips of different size and you want to write directly the
  result of the concatenation to a file, use the method "compose" instead.

- method="compose", if the clips do not have the same
  resolution, the final resolution will be such that no clip has
   to be resized.
   As a conseque